In [ ]:
from pythreejs import *
import ipywidgets
from IPython.display import display

In [ ]:
view_width = 800
view_height = 600

In [ ]:
sphere = Mesh(
    SphereBufferGeometry(1, 32, 16),
    MeshStandardMaterial(color='red')
)

In [ ]:
cube = Mesh(
    BoxBufferGeometry(1, 1, 1),
    MeshPhysicalMaterial(color='green'),
    position=[2, 0, 4]
)

In [ ]:
plane = Mesh(
    PlaneBufferGeometry(10, 10),
    MeshPhysicalMaterial(color='gray'),
    position=[0, -2, 0],)
plane.rotation = (-3.14/2, 0, 0, 'XYZ')

In [ ]:
camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = SpotLight(position=[0, 10, 10], angle = 0.3, penumbra = 0.1)
key_light.target = cube
ambient_light = AmbientLight()

In [ ]:
scene = Scene(children=[sphere, cube, plane, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True)

In [ ]:
renderer

In [ ]:
# Turn on shadows in the renderer
renderer.shadowMap.enabled = True
renderer.shadowMap.type = 'PCFSoftShadowMap'  # default PCFShadowMap

In [ ]:
# Enable shadows for the light
key_light.castShadow = True
# Enable objects for casting/receiving shadows:
sphere.castShadow = True
cube.castShadow = True
plane.receiveShadow = True

In [ ]:
cube.position = (0, 1, 2)

In [ ]:
# Also enable sphere to receive shadow:
sphere.receiveShadow = True
sphere.material.needsUpdate = True

In [ ]:
key_light.shadow.mapSize = (2048, 2048)

In [ ]:
repr(key_light.shadow.camera)